## MSELoss Functionality Test

Network Class

In [1]:
import torch.nn as nn
from collections import OrderedDict

class NN(nn.Module):

    def __init__(self, input_size, middle_width, num_classes):


        super(NN, self).__init__()
        self.features = nn.Sequential(OrderedDict([
            ('hidden_layer', nn.Linear(input_size, middle_width)),
            ('hidden_activation', nn.ReLU()),
        ]))
        self.readout = nn.Linear(middle_width, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.readout(x)

        return x

Network Functions

In [2]:
import torch  # Base torch library
from torch.utils.data import DataLoader  # Minibathces
import torchvision.datasets as datasets  # MNIST dataset
import torchvision.transforms as transforms
import numpy as np
from torch.nn.functional import one_hot


def set_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return device


def mnist_dataset(batch_size, train=True, values=list(range(10))):
    # Initializing MNIST data set.
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

    targets_list = dataset.targets.tolist()
    values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

    # Creating a subset of ### MNIST targets.
    subset = torch.utils.data.Subset(dataset, values_index)
    loader = DataLoader(dataset=subset, batch_size=batch_size, shuffle=True)

    return loader


def train(loader, device, model, loss_function, optimizer_function, values=list(range(10))):
    # Training on each data point.

    # Set array full of zeros.
    kernel_alignments = torch.zeros(len(loader))

    for batch_idx, (data, targets) in enumerate(loader):
        data = data.reshape(data.shape[0], -1).to(device=device)
        targets = targets.to(torch.float32).to(device=device)

        # Forwards.
        scores = model(data)
        # loss = loss_function(scores, classify_targets(targets, values))

        labels = one_hot(targets.long() % len(values)).to(torch.float32)
        output = loss_function(scores, labels)

        # Backwards.
        optimizer_function.zero_grad()
        output.backward()

        optimizer_function.step()
        phi = model.features(data)
        print("PHI:")
        print(phi.size())
        print("TARGETS:" )
        print(targets.size())

        kernel_alignments[batch_idx] = kernel_calc(targets, phi)

    # return torch.mean(kernel_alignments).item(), torch.std(kernel_alignments).item()/len(kernel_alignments)
    # return mean and STD or STE of kernel alignment


def record_accuracy(device, model, train_loader, test_loader, epoch, ste, mean, values=list(range(10))):
    epoch_accuracy = np.array([[
        epoch + 1,
        check_accuracy(device, model, train_loader, values).cpu(),
        check_accuracy(device, model, test_loader, values).cpu(),
        mean,
        ste
    ]])

    return epoch_accuracy


def check_accuracy(device, model, loader, values=list(range(10))):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = classify_targets(y, values).to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            # 64images x 10,

            predictions = scores.argmax(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    return num_correct / num_samples


def classify_targets(targets, values):
    new_targets = targets.clone()

    # Changing targets to a classifiable number.
    for key, element in enumerate(values):
        new_targets[targets == element] = key
    return new_targets


# Kernel Alignment Fucntions

def kernel_calc(y, phi):

    # Output Kernel
    y = torch.t(torch.unsqueeze(y, -1))
    K1 = torch.matmul(torch.t(y), y)
    K1c = kernel_centering(K1.float())

    # Feature Kernel
    K2 = torch.mm(phi, torch.t(phi))
    K2c = kernel_centering(K2)

    return kernel_alignment(K1c, K2c)


def frobenius_product(K1, K2):
    return torch.trace(torch.mm(K2, torch.t(K1)))


def kernel_alignment(K1, K2):
    return frobenius_product(K1, K2) / ((torch.norm(K1, p='fro') * torch.norm(K2, p='fro')))


def kernel_centering(K):
    # Lemmna 1

    m = K.size()[0]
    I = torch.eye(m)
    l = torch.ones(m, 1)

    # I - ll^T / m
    mat = I - torch.matmul(l, torch.t(l)) / m

    return torch.matmul(torch.matmul(mat, K), mat)


def ones(vector):
    for i in range(vector.size()[1]):
        if vector[0][i] == 9:
            vector[0][i] = 1
        elif vector[0][i] == 8:
            vector[0][i] = -1
    return vector


In [61]:
targets = torch.randint(low = 7, high = 9, size=(5,))
targets

tensor([7, 7, 8, 7, 8])

In [75]:
i = targets.unique()
for value in i:
    print(value.item())

7
8


In [62]:
classify_targets(targets, [0, 1])

tensor([7, 7, 8, 7, 8])

Traing the model

In [3]:
# Setting device
device = set_device()

In [4]:
# Initializing the model
model = NN(784, 12, 2)

In [5]:
# Loading MNIST values
mnist_values = [2, 7]
print(f"MNIST digits {mnist_values}")
train_loader = mnist_dataset(7, values=mnist_values)
validate_loader = mnist_dataset(7, train=False, values=mnist_values)

MNIST digits [2, 7]


NameError: name 'unique' is not defined

In [6]:
# Loss function
loss = nn.MSELoss() #!! REPLACE W/ MSELOSS

In [7]:
# Optimizer
import torch.optim as optim
optimizer = optim.SGD([{'params': model.features.hidden_layer.parameters()},
                          {'params': model.readout.parameters(),
                           'lr': 0.1}],
                         lr=0.01)

In [8]:
# Creating an empty array for storing accuracy metrics
import numpy as np
accuracy = np.zeros((5, 5))

In [46]:
len(train_loader)

1747

In [43]:
# Training
for epoch in range(5):

    print(epoch)
    # Slow Model
    phi, targets = train(train_loader, device, model, loss, optimizer, values=mnist_values)
    accuracy[epoch][0] = epoch + 1
    accuracy[epoch][1] = check_accuracy(device, model, train_loader, mnist_values).cpu()
    accuracy[epoch][2] = check_accuracy(device, model, validate_loader, mnist_values).cpu()
    #accuracy[epoch][3] = sl_mean
    #accuracy[epoch][4] = sl_ste
    # CALCULATE THE K.A. AND RECORD IT TO A CSV (FOR SLOW MODEL)
    print("Slow: ")
    print(accuracy[epoch])

    print(f"-Finished epoch {epoch + 1}/5")

0
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7]

/Users/cameronkaminski/anaconda3/envs/slow-feature-learning/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])
PHI:
torch.Size([7, 12])
TARGETS:
torch.Size([7])


KeyboardInterrupt: 

In [10]:
targets.size()

torch.Size([1])

In [11]:
phi.size()

torch.Size([1, 12])

In [12]:
# Accuracy csv
import pandas as pd
complete_dataframe = pd.DataFrame(accuracy).to_csv('accuracy_metrics')

Fixing MNSIT Dataset

In [13]:
def mnist_dataset(batch_size, train=True, values=list(range(10))):
    # Initializing MNIST data set.
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

    targets_list = dataset.targets.tolist()
    values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

    # Creating a subset of ### MNIST targets.
    subset = torch.utils.data.Subset(dataset, values_index)
    loader = DataLoader(dataset=subset, batch_size=batch_size, shuffle=True)

    return subset

In [14]:
dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

In [15]:
values = [7]
targets_list = dataset.targets.tolist()
values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

In [16]:
i = 0
for i in range(11):
    values = [i]
    targets_list = dataset.targets.tolist()
    values_index = [j for j in range(len(dataset)) if targets_list[j] in i]
    print(values)
    print (len(values_index))

TypeError: argument of type 'int' is not iterable

In [ ]:
ls = [1]
ls

[1]

In [ ]:
# Creating a subset of ### MNIST targets.
subset = torch.utils.data.Subset(dataset, values_index)
loader = DataLoader(dataset=subset, batch_size=batch_size, shuffle=True)

NameError: name 'batch_size' is not defined

In [ ]:
for batch_idx, (data, targets) in enumerate(data_set):
    # print("DATA: ")
    # print(data)
    print("TARGETS")
    print(targets)

TARGETS
5
TARGETS
0
TARGETS
4
TARGETS
1
TARGETS
9
TARGETS
2
TARGETS
1
TARGETS
3
TARGETS
1
TARGETS
4
TARGETS
3
TARGETS
5
TARGETS
3
TARGETS
6
TARGETS
1
TARGETS
7
TARGETS
2
TARGETS
8
TARGETS
6
TARGETS
9
TARGETS
4
TARGETS
0
TARGETS
9
TARGETS
1
TARGETS
1
TARGETS
2
TARGETS
4
TARGETS
3
TARGETS
2
TARGETS
7
TARGETS
3
TARGETS
8
TARGETS
6
TARGETS
9
TARGETS
0
TARGETS
5
TARGETS
6
TARGETS
0
TARGETS
7
TARGETS
6
TARGETS
1
TARGETS
8
TARGETS
7
TARGETS
9
TARGETS
3
TARGETS
9
TARGETS
8
TARGETS
5
TARGETS
9
TARGETS
3
TARGETS
3
TARGETS
0
TARGETS
7
TARGETS
4
TARGETS
9
TARGETS
8
TARGETS
0
TARGETS
9
TARGETS
4
TARGETS
1
TARGETS
4
TARGETS
4
TARGETS
6
TARGETS
0
TARGETS
4
TARGETS
5
TARGETS
6
TARGETS
1
TARGETS
0
TARGETS
0
TARGETS
1
TARGETS
7
TARGETS
1
TARGETS
6
TARGETS
3
TARGETS
0
TARGETS
2
TARGETS
1
TARGETS
1
TARGETS
7
TARGETS
9
TARGETS
0
TARGETS
2
TARGETS
6
TARGETS
7
TARGETS
8
TARGETS
3
TARGETS
9
TARGETS
0
TARGETS
4
TARGETS
6
TARGETS
7
TARGETS
4
TARGETS
6
TARGETS
8
TARGETS
0
TARGETS
7
TARGETS
8
TARGETS
3
TARGETS
1
